In [2]:
import re

def label_node(ckt_graph,input_vars,key):
    if key in input_vars:
        ckt_graph[key]["level"] = 0
        return 1
    elif ckt_graph[key]["level"] != -1:
        return ckt_graph[key]["level"]+1
    else:
        max_level = -1 
        for input in ckt_graph[key]["inputs"]:
            temp_level = label_node(ckt_graph,input_vars,input)
            if temp_level>max_level:
                max_level = temp_level
        ckt_graph[key]["level"] = max_level
        return max_level+1

def bench2graph(benchfile):
    with open(benchfile, 'r') as file:
        lines = file.readlines()
    
    ckt_graph={}
    output_vars=[]
    input_vars=[]
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)
            if var_name in ckt_graph.keys():
                ckt_graph[var_name]["gate"] = gate_match[1]
                ckt_graph[var_name]["inputs"] = gate_match[2:]
            else:  
                node={}  
                node["level"] = -1
                node["outbound"] = []
                node["gate"] = gate_match[1]
                node["inputs"] = gate_match[2:]
                ckt_graph[var_name] = node
            for inp_node in gate_match[2:]:
                if inp_node in ckt_graph.keys():
                    ckt_graph[inp_node]["outbound"].append(var_name)
                else:
                    node={}
                    node["level"] = -1
                    node["outbound"] = [var_name]
                    node["gate"] = ""
                    node["inputs"] = []
                    ckt_graph[inp_node] = node
                    
        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    output_vars.append(match.group(1))
    
    for key in ckt_graph.keys():
        if key in output_vars:
            label_node(ckt_graph,input_vars,key)

    return ckt_graph,input_vars,output_vars,wires


def graph2bench(ckt_node,inputs,outputs,bench_name,input_node =False):
    bench=[f"# {bench_name}",f"# {len(inputs)} INPUTS", f"# {len(outputs)} OUTPUTS", f"# {len(ckt_node.keys())} Gates", "\n"]

    for input in inputs:
        bench.append(f"INPUT({input})")
    for output in outputs:
        bench.append(f"OUTPUT({output})")
    bench.append("\n")

    gate_no = 0
    key_len = len(ckt_node.keys())
    if input_node:
        key_len -= len(inputs)
    appending_level = 1
    while gate_no<key_len:
        for key in ckt_node.keys():
            if ckt_node[key]["level"]==appending_level:
                if len(ckt_node[key]["inputs"])>1:
                    gate_inp = ", ".join(ckt_node[key]["inputs"])
                else: gate_inp = ckt_node[key]["inputs"][0]
                gate_name = ckt_node[key]["gate"]
                bench.append(f"{key} = {gate_name}({gate_inp})")
                gate_no += 1
        appending_level += 1

    with open(bench_name+".bench", 'w') as file:
        file.write("\n".join(bench))
        print(bench_name+" bench file created")
                

In [3]:
import re
def nandlib_synth2bench(file):
    with open(file, 'r') as file_name:
        file_content = file_name.read()
    file_content = file_content.replace("\n","")
    file_content = file_content.split(";")

    gate_pattern = r'\b(AND|OR|NAND|NOR|XOR|XNOR|INV)'
    pin_pattern = r'\(\s*([^\(\)]+?)\s*\)'
    gates=[]
    for line in file_content:
        line = line.strip()
        gate = re.findall(gate_pattern, line)
        if gate:
            if gate[0] == "INV": gate[0] = "NOT"
            line = line.replace("K[","keyinput")
            line = line.replace("]","")
            line = line.replace("PPI[","in_")
            line = line.split(".Z")
            in_pin = re.findall(pin_pattern, line[0])
            out_pin = re.findall(pin_pattern, line[1])
            if len(in_pin)>1:
                inpin = ", ".join(in_pin)
            else:
                inpin = in_pin[0]
            gates.append(f"{out_pin[0]} = {gate[0]}({inpin})")

    
    return gates

In [ ]:

file = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/obfuscated/cac48"
gates = nandlib_synth2bench(file+".v")

inp=["INPUT(OPO)","OUTPUT(LPO)"]
key=[]
for i in range(48):
    inp.append(f"INPUT(in_{i})")
    key.append(f"INPUT(keyinput{i})")


with open(file+".bench", 'w') as file_name:
    file_name.write("\n".join(inp+key+gates))
    print(" bench file created")

ckt_graph,input_vars,output_vars,wires = bench2graph(file+".bench")
graph2bench(ckt_graph,input_vars,output_vars,file,True)

In [2]:

def convert_gate_line(line, gate_count):
    match = re.findall(r'\b\w+\b', line)
    gate_dict=[ "AND", "NAND", "OR", "NOR", "XOR", "XNOR", "NOT", "BUF", "DFF"]
    
    if match:
        output_var = match[0]
        gate_type = match[1]  # Convert gate type to lowercase
        inputs = match[2:]
        
        if gate_type in gate_dict:
            gate_unique_name = f"{gate_type}{len(inputs)}_{gate_count}"
            converted_line = f"{gate_type.lower()} {gate_unique_name} ({output_var}, {', '.join(inputs)});"
            return gate_type, converted_line
    
    return None, None


def bench2verilog(file_path, file_name,des_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gate_count = 1
    input_vars = []
    output_vars = []
    assigned_vars = []
    dff_module = [
            "module DFF(Q, clk, D);",
            "input D;",
            "input clk;",
            "output Q;",
            "always @(clk)", 
            "begin",
            "  Q <= D;", 
            "end", 
            "endmodule"]
    opr_lines=[]
    gate_types=[]

    # Regular expressions to match INPUT, OUTPUT, and gate assignments
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    assign_pattern = re.compile(r'^(\w+)\s*=\s*(NAND|NOR|AND|OR|XOR|XNOR|NOT|DFF|BUF)\((.*)\)')

    for line in lines:
        line = line.strip()
        if line.startswith("INPUT"):
            match = input_pattern.match(line)
            if match:
                input_vars.append(match.group(1))
        elif line.startswith("OUTPUT"):
            match = output_pattern.match(line)
            if match:
                output_vars.append(match.group(1))
        else:
            match = assign_pattern.match(line)
            if match:
                assigned_vars.append(match.group(1))
                gate_type,opr_line = convert_gate_line(line,gate_count)
                if opr_line is not None:
                    opr_lines.append(opr_line)
                    if gate_type not in gate_types:
                        gate_types.append(gate_type) 
                    gate_count += 1

    # Intermediate variables are those assigned but not inputs or outputs
    intermediate_vars = [var for var in assigned_vars if var not in input_vars and var not in output_vars]

    module_heading = f"module {file_name}({','.join(input_vars + output_vars)});"
    input_line = f"input {','.join(input_vars)};"
    output_line = f"output {','.join(output_vars)};"
    wire_line = f"wire {','.join(intermediate_vars)};"
    termination_line = "endmodule"
    main_module="\n".join([module_heading,"",input_line,output_line,wire_line,""]+opr_lines+["",termination_line])
    
    if "DFF" in gate_types:
        main_module = "\n".join(dff_module + ["\n", main_module])

    des_file = os.path.join(des_path,file_name+".v")
    with open(des_file, 'w') as file:
        file.write(main_module)
        print(file_name+" verilog file created")


In [ ]:
import os,re
directory = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/verilog_ckt/queue"
des_path = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/verilog_ckt/obfuscated"
for dirpath, _, filenames in os.walk(directory):
    for filename in filenames:
        if ".bench" in filename: 
            file_path = os.path.join(dirpath,filename)
            filename = filename.split(".")[0]
            bench2verilog(file_path,filename,des_path)

In [ ]:
#ttlock/cac injector
import random
ckt_name = "c1355"
bit_no = "8"
org_file = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\bench_ckt\\{ckt_name}.bench"
lock_file = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\obfuscated\\cac{bit_no}.bench"
file_name = f"D:\\Personal\\New folder\\BUET_SAT_SIMULATOR\\8bit\\{ckt_name}_cac_8k.bench"

def bench2list(org_file, add_out=True):
    with open(org_file, 'r') as file:
            lines = file.readlines()
        
    output_vars=[]
    input_vars=[]
    io_lines = []
    gate_lines = []
    wires = []
    input_pattern = re.compile(r'^INPUT\((\w+)\)')
    output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
    
    for line in lines:
        line = line.strip()
        
        if "=" in line:
            gate_lines.append(line)
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            wires.append(var_name)

        elif len(line)>3:
            if line.startswith("INPUT"):
                match = input_pattern.match(line)
                if match:
                    input_vars.append(match.group(1))
                    io_lines.append(line)
            elif line.startswith("OUTPUT"):
                match = output_pattern.match(line)
                if match:
                    if add_out:
                        io_lines.append(line)
                    output_vars.append(match.group(1))
                    
    
    return gate_lines,io_lines,input_vars,output_vars,wires

org_gates, org_io, org_inputs, org_outputs, org_wires = bench2list(org_file)
lock_gates, lock_io, lock_inputs, lock_outputs, lock_wires = bench2list(lock_file, False)

bench = org_io
for i in range(32):
    bench.append(f"INPUT(keyinput{i})")

gate_index = org_wires.index(org_outputs[-1])
org_gates[gate_index] = org_gates[gate_index].replace(org_outputs[-1],org_outputs[-1]+"_enc")
lock_gates[-1] = lock_gates[-1].replace(lock_outputs[0],org_outputs[-1])
lock_gates[-3] = lock_gates[-3].replace("OPO",org_outputs[-1]+"_enc")

wirex=[]
for i in range(len(lock_gates)):
    if "in_" in lock_gates[i]:
        gate_match = re.findall(r'\b\w+\b', lock_gates[i])
        for inp in gate_match[2:]:
            if "in_" in inp:
                inp_index = int(inp.split("_")[-1].strip())
                lock_gates[i] = lock_gates[i].replace(inp,org_inputs[inp_index])
    

bench += org_gates+lock_gates
with open(file_name, 'w') as file:
    file.write("\n".join(bench))
    print(file_name+" bench file created")

In [12]:
import re 

org_file= "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/bench_ckt/C6288.bench"
with open(org_file, 'r') as file:
    lines = file.readlines()

input_pattern = re.compile(r'^INPUT\((\w+)\)')
output_pattern = re.compile(r'^OUTPUT\((\w+)\)')
assign_pattern = re.compile(r'^(\w+)\s*=\s*(NAND|NOR|AND|OR|XOR|XNOR|NOT|DFF|BUF)\((.*)\)')

new_bench=[]

for line in lines:
    line = line.strip()
    if line.startswith("INPUT"):
        match = input_pattern.match(line)
        if match:
            new_bench.append(f"INPUT(G{match.group(1)}gat)")
    elif line.startswith("OUTPUT"):
        match = output_pattern.match(line)
        if match:
            new_bench.append(f"OUTPUT(G{match.group(1)}gat)")
    elif "=" in line:
            gate_match = re.findall(r'\b\w+\b', line)
            var_name = gate_match[0].strip()
            gate_name = gate_match[1].strip()
            inps = []
            for gate in gate_match[2:]: inps.append(f"G{gate}gat")
            inps = ", ".join(inps)
            new_bench.append(f"G{var_name}gat = {gate_name}({inps})")
    else: new_bench.append(line)

org_file = org_file.replace(".bench","_new.bench")
with open(org_file, 'w') as file:
    file.write("\n".join(new_bench))


In [ ]:
import algo_methods

In [ ]:
org_ckt_ls = "/home/mahmudul-hasan/Research/BUET_SAT_SIMULATOR/c432/c432.bench"
key_str = "10011000111110101001100011111010"

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_sarlock_32k.bench"
algo_methods.sarlock(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c6288/c6288_rll_32k.bench"
algo_methods.RLL(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "src/c6288_antisat_32k.bench"
algo_methods.libar(org_ckt_ls,obs_ckt_ls,key_str,libar_percent=0.6)

In [ ]:
obs_ckt_ls = "/Users/mahmudulhasan/Downloads/BUET_SAT_SIMULATOR/c432/c432_antisat_32k.bench"
algo_methods.anti_sat(org_ckt_ls,obs_ckt_ls,key_str)

In [ ]:
obs_ckt_ls = "src/c6288_antisat_32k.bench"
algo_methods.asob(org_ckt_ls,obs_ckt_ls,key_str,no_rll_keybit=6)

In [4]:
org_ckt_ls = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/bench_ckt/c432.bench"
obs_ckt_ls = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/obfuscated_ckt/k8/c432_libar_8k_unrolled.bench"

In [27]:
import algo_methods
org_ckt_ls = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/bench_ckt/c1355.bench"
obs_ckt_ls = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/obfuscated_ckt/libars/c1355_libar_32k_unrolled.bench"

In [25]:
algo_methods.sat(org_ckt_ls,obs_ckt_ls)

"Didn't Found Any Satisfiable Key; func_iteration= 10; func_exe_time= 0.08394598960876465; nonfunc_exe_time= 0\n"

In [28]:
algo_methods.appsat(org_ckt_ls,obs_ckt_ls)

"Didn't Found Any Satisfiable Key; func_iteration= 16; func_exe_time= 20.170973777770996; nonfunc_exe_time= 0\n"

In [22]:
algo_methods.hamming_sweep(org_ckt_ls,obs_ckt_ls,max_iter=1000)

"Didn't Found Any Satisfiable Key; func_iteration= 16; func_exe_time= 0.27564573287963867; nonfunc_exe_time= 0\n"

In [1]:
import algo_methods
import math
key8 = "10011001"
key16 = "1110100011101000"
key32 = "10011000111110101001100011111010"

key5 =  "10011"
key10 = "1110100010"
key15 = "100110001111101"
key20 = "00011011111100101001"



ckt = "c1"
bit_no = 20
key = key20
abs_path = "/home/mahmudul-hasan/Research"


src = f"{abs_path}/BUET_SAT_SIMULATOR/bench_ckt/{ckt}.bench"
#rll= f"{abs_path}/BUET_SAT_SIMULATOR/{bit_no}bit/{ckt}_rll_{bit_no}k.bench"
rll= f"{abs_path}/BUET_SAT_SIMULATOR/rll_libar/{ckt}_rll_{bit_no}k.bench"

#algo_methods.RLL(src,rll,key)


In [ ]:
keys=["1100011010101100","001010101111000101011101","10011000111110101010111101010101"]#,"111111111001010101000000000011111101010111110101"]
rll_c1908 = ["0100101001111100", "000110110111000101011101", "00011101100100101010111101010101"]
#"c432" "c1355" "c3540" success
ckt = "c6288"
abs_path = "/Users/mahmudulhasan/Documents"
src = f"{abs_path}/BUET_SAT_SIMULATOR/bench_ckt/{ckt}.bench"

for key in keys: 
    bit_no = len(key)
    rll= f"{abs_path}/BUET_SAT_SIMULATOR/obfuscated_ckt/{ckt}_rll_{bit_no}k.bench"
    result = algo_methods.sat(src,rll)

    if 'key= ' in result:
        rkey = (result.split('key= ')[1].split(';')[0]).strip()
    else: 
        rkey =""
    print(rkey)
    print(key)
    if key == rkey:
        print("success")

In [ ]:
ckt = "c1908"

ovlp_pins =[0] 
libar_prcnt=[2,5,10,15,25,50,75,100]
keys = ["10011","1110100010","100110001111101","10011000111110101001"]

for key in keys:
    bit_no = len(key) 
    rll= f"{abs_path}/BUET_SAT_SIMULATOR/rll_libar/{ckt}_rll_{bit_no}k.bench"
    for ovlp_pin in ovlp_pins:
        for prcnt in libar_prcnt:
            libar_no = math.ceil(prcnt*int(bit_no)/100)
            algo_methods.libar(rll,f"{abs_path}/BUET_SAT_SIMULATOR/libars/{ckt}_{bit_no}k_9pinovlp_libar_{prcnt}.bench",key,libar_no,True,ovlp_pin)

In [73]:
import os 
directory = os.path.join(abs_path,f"BUET_SAT_SIMULATOR/libars")
for dirpath, _, filenames in os.walk(directory):
    for filename in filenames:
        if "unrolled.bench" not in filename: 
            file_path = os.path.abspath(os.path.join(dirpath, filename))
            os.remove(file_path)
        

In [6]:
#result.csv process file

import csv,os


# Function to process each line
def process_line(line):
    parts = line.split(' ', 1)  # Split into circuit name and the rest
    circuit_name = parts[0]
    new_parts = parts[1].split(':') # Split into attack method and the rest
    attack = (new_parts[0].split(" ")[0]).strip()
    result_part = new_parts[1].strip()

    # Initialize default values
    result = "Failed"
    iteration = "-"
    time = "-"
    key = "-"

    #extract value from circuit name
    c1_row = circuit_name.split("_")
    ckt = c1_row[0].strip()
    lock_algo = c1_row[1].strip()
    key_bit = c1_row[2].split(".")[0]
    result_part = result_part.lower()
    # Check if a key was found
    if "func_iteration" in result_part:
        #key = (result_part.split('key= ')[1].split(';')[0]).strip()
        #key = "\'"+key
        #keys_dict = {5:"10011",10:"1110100010",15:"100110001111101",20:"10011000111110101001"}
        #keys_dict = {16:"1100011010101100",24:"001010101111000101011101",32:"10011000111110101010111101010101",48:"111111111001010101000000000011111101010111110101"}
        #if key == org_key:
        iteration = result_part.split('func_iteration= ')[1].split(';')[0]
        time = result_part.split('func_exe_time= ')[1].split(';')[0]
        if "key=" in result_part:
            key = (result_part.split('key= ')[1].split(';')[0]).strip()
            key = "\'"+key 
            result = "Success"
        elif "memory" in result_part:
            result = "Memory Limit Crossed"
        elif "time limit" in result_part:
            result = "time limit crossed"
    else: 
        result = "time limit crossed"
        iteration = 2001
        time = 3*3600
        
        
    
    return [ckt, lock_algo, key_bit, attack, result, key, iteration, time]



project_path = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR"
txt_file = os.path.join(project_path,"src/raw_rslt_nlibar.txt")
csv_file = os.path.join(project_path,"src/final_result.csv")

with open(txt_file, 'r') as file:
    lines = file.readlines()
# Process all lines
processed_lines = [["ckt","Lock Method","key_bit","Attack Method","Key Found","key","Iteration","Time (s)"]]

done_list = []
for line in lines:
    line = line.strip()
    if len(line)>10:
        row_line = process_line(line)
        processed_lines.append(row_line)
        done_list.append("".join(row_line[:4]))


#for final files only ============>>>>>>>>>>>>>>>>
with open(csv_file, 'r') as infile:
    reader = csv.reader(infile)
    rows = [next(reader)]  # Read the header if the CSV has one
    

    for row in reader:
        row_key = "".join(row[:4])
        if row_key in done_list:
            index = done_list.index(row_key)
            rows.append(row[:4]+processed_lines[index][4:])
        else: rows.append(row)

    processed_lines = rows

    print(len(processed_lines))
#<<<<<<<<<<<<<<=====================================


# Write or append to the result.csv file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for row in processed_lines:
        writer.writerow(row)

print("Data written to result.csv successfully!")


241
Data written to result.csv successfully!


In [1]:
#final result duplicate remover
src_file = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/src/final_result.txt"
with open(src_file,"r") as file:
    lines = file.readlines()

lines = set(lines)

with open(src_file,"w") as file:
    file.writelines(lines)

In [6]:
import csv,os
avg_dict = {
    'c432': {
        'libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'rll': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}}
    },
    'c1355': {
        'libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'rll': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}}
    },
    'c1908': {
        'libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'rll': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}}
    },
    'c3540': {
        'libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock+libar': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'sarlock': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'cac': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'antisat': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}},
        'rll': {'8k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '16k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '24k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}, '32k': {'time': 0, 'iter': 0, 'pass': 0, 'count': 0}}
    }
}


project_path = "/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR"
in_csv = os.path.join(project_path,"src/result_hlibar.csv")
out_csv = os.path.join(project_path,"src/hlibar_avg.csv")

# Step 1: Read the CSV file
with open(in_csv, 'r') as infile:
    reader = csv.reader(infile)
    header = next(reader)  # Read the header if the CSV has one
    
    # Store the rows for modification
    rows = [['ckt','bit_no','defence','avg_time','avg_iteration','success_rate']]
    for row in reader:
        avg_dict[row[0]][row[1]][row[2]]['count'] += 1
        if row[7] != '-':
            avg_dict[row[0]][row[1]][row[2]]['time'] += float(row[7])
            avg_dict[row[0]][row[1]][row[2]]['iter'] += float(row[6])
            if row[4] == "Success":  avg_dict[row[0]][row[1]][row[2]]['pass'] += 1
    
    for ckt in avg_dict.keys():
        for defence in avg_dict[ckt].keys():
            for bit_no in avg_dict[ckt][defence].keys():
                if avg_dict[ckt][defence][bit_no]["count"]>0:
                    avg_time = avg_dict[ckt][defence][bit_no]["time"]/avg_dict[ckt][defence][bit_no]["count"]
                    avg_iteration = avg_dict[ckt][defence][bit_no]["iter"]/avg_dict[ckt][defence][bit_no]["count"]
                    success_rate = avg_dict[ckt][defence][bit_no]["pass"]/avg_dict[ckt][defence][bit_no]["count"]

                    rows.append([ckt,bit_no,defence,avg_time,avg_iteration,success_rate])


# Step 2: Write the modified data to a new CSV file
with open(out_csv, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    
    
    # Write the modified rows
    writer.writerows(rows)


In [3]:
from pathlib import Path
import re
from pathlib import Path


# Define the folder path
folder_path = Path("/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/hlibar")
txt_path = Path("/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/src/raw_rslt6.txt")


def get_executed_list(file_path):
    pattern = r'\b\w+[\w+]*\.bench\b'
    file_names = set()
    with open(file_path, 'r') as file:
        for line in file:
            matches = re.findall(pattern, line)
            if matches:
                file_names.update(matches)

    return list(file_names)


done_list = get_executed_list(txt_path)
print(len(done_list))
queue_list = []
file_paths = [file for file in folder_path.rglob('*') if file.is_file()]
print(len(file_paths))
# Print the file name and absolute path
for file in file_paths:
    if file.name not in done_list:
        #print(f"File Name: {file.name}, Absolute Path: {file.resolve()}")
        queue_list.append(file.name)
queue_list = list(set(queue_list))
print(len(queue_list))
with open("/Users/mahmudulhasan/Documents/BUET_SAT_SIMULATOR/src/queue.txt", 'w') as file:
        file.write(",".join(queue_list))


48
48
0
